In [5]:
from pyspark import SparkContext
import sys

start = '1'
LINECOUNT = 1
Nodes = {}
class Node:
    def __init__(self,id):  
        global LINECOUNT
        self.id = id
        self.Children = []
        if id == start:
            self.distance = 0
        else:
            self.distance = LINECOUNT

        
    def AddChild(self,cid):
        self.Children.append(cid)
        
    @staticmethod
    def get_node(id):
        if id not in Nodes:
            Nodes[id] = Node(id)
        return Nodes[id]
    
class Mapper:
    def __init__(self):
        pass
    
    def map(self,next_nodes):
        sc = SparkContext.getOrCreate()
        next_nodes = sc.parallelize(next_nodes) 
        return next_nodes.flatMap(lambda nid:self.map_func(nid,Nodes))
        
    def map_func(self,nid,Nodes):
        node = Nodes[nid]
        cs = node.Children
        dis_list = [(cid,node.distance+1) for cid in cs]
        for cid in cs:
            dis_list.append((cid,Nodes[cid].distance ))
        return dis_list

class Reducer:
    def __init__(self):
        pass
    
    def reduce(self,dis_map):
        return dis_map.groupByKey().map(lambda x:(x[0],min(x[1])))

class FileProcessor:
    def __init__(self,fname):
        self.set_node_list(fname)
    
    def set_node_list(self,fname):
        global LINECOUNT
        ifile = open(fname)
        lines = ifile.readlines()
        LINECOUNT = len(lines)
        for line in lines:
            if len(line) < 1:
                continue
            line = line.split(' ')
            p = Node.get_node(line[0].strip())
            cid = line[1].strip()
            child = Node.get_node(cid)
            p.AddChild(cid)
            
            
class BFS:
    def __init__(self,fname):
        fp = FileProcessor(fname)
    
    def find_distance(self,Nodes,start,end):
        next_nodes = [start]
        Found = False
        for i in range(len(Nodes)-1):
            new_next_nodes = []
            for nid in next_nodes:
                node = Node.get_node(nid)
                for cid in node.Children:
                    new_next_nodes.append(cid)
            new_next_nodes = list(set(new_next_nodes))
            mapper = Mapper()
            dis_map = mapper.map(next_nodes)
            reducer = Reducer()
            mdis = reducer.reduce(dis_map)
            for (id,dis) in mdis.collect():
                Nodes[id].distance = dis
                if id == end:
                    Found = True
                    break
            if Found:
                break
            print(Nodes[end].distance)
            next_nodes = new_next_nodes[:]
        print(Nodes[end].distance)
        
    def find_shortest_paths(self,test_file):
        global Nodes
        cnodes = Nodes.copy()
        for l in open(test_file):
            l = l.strip()
            if len(l) == 0:
                continue
            l = l.split(' ')
            start = l[0]
            end = l[1]
            bfs.find_distance(cnodes,start,end)

bfs = BFS('direct_small.graph')
bfs.find_shortest_paths('DTest')

508837
508837
508837
508837
508837
508837


KeyboardInterrupt: 